In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import sys
from Function import Function
from losses import MSE
from activations import TanhActivation, SigmoidActivation

from load_data import get_train_test_loaders

from torch.nn.modules.loss import CrossEntropyLoss

%load_ext autoreload
%autoreload 2

In [ ]:
train_loader,test_loader, train_size, test_size = get_train_test_loaders(binary=False)

def get_minibatches(loader, device):
    for data, target in loader:
        normalize = torch.nn.BatchNorm2d(1)
        data = normalize(data)
        #import pdb;pdb.set_trace()
        
        data = torch.squeeze(data)
        target = target.cuda(async=True)
        data = data.cuda(async=True)
        
        batch_n = data.size()[0]
        X = data.view(batch_n, -1)
        ones = torch.ones((X.size()[0], 1), device=device)
        X = torch.cat((X, ones), 1)
        
        y_onehot = torch.zeros((target.size()[0], 10), device=device)
        y_onehot.zero_()
        y_onehot.scatter_(1, target.view(-1,1), 1)
        
        X = torch.autograd.Variable(X)
        y = torch.autograd.Variable(y_onehot)
        yield X.t(), y

In [ ]:
device = torch.device('cuda')
dtype = torch.float

layers_dims = [28*28 + 1, 128, 10]
batch_N = 128

activation1 = TanhActivation() # Tanh
activation2 = SigmoidActivation() # Sigmoid

loss_function = CrossEntropyLoss()
function = Function()

w1 = torch.randn(layers_dims[1], layers_dims[0], dtype=dtype, device=device, requires_grad=True)
w2 = torch.randn(layers_dims[2], layers_dims[1], dtype=dtype, device=device, requires_grad=True)

with torch.no_grad():
    w1 *= 0.01
    w2 *= 0.01

learning_rate = 1e-7
for t in range(50):
    
    for i,(X, y) in enumerate(get_minibatches(train_loader, device)):
        y_h1 = function.forward(w1, X)
        y_h2 = activation1.forward(y_h1)
        y_h3 = function.forward(w2, y_h2)
        y_pred = activation2.forward(y_h3).t()
        
        # replace nans with 0
        y_pred = torch.where(torch.isnan(y_pred), torch.zeros(1, device=device, dtype=dtype), y_pred)
                             
        loss = loss_function(y_pred, y.argmax(1))
        print(t, loss.item())

        loss.backward()
        
        with torch.no_grad():
            #import pdb; pdb.set_trace()
            w1 -= learning_rate * w1.grad
            w2 -= learning_rate * w2.grad

            # Manually zero the gradients after running the backward pass
            w1.grad.zero_()
            w2.grad.zero_()

In [ ]:
#torch.nn.init.constant_(tensor, 1)
torch.tensor([0.01], device=device, dtype=dtype).expand(y.shape)

In [ ]:
w1